# The functions for tran_ncb, predict_ncb(using Decrete algo for continuous value)

In [26]:
#calc conditional probility for continus attri
#1 bins
# take 2 files as input?
# have to use Laplace recorrect , next time
LABEL_LIST=["low", "median","high"]

def calc_condi_prob(X, y):
    # rows and cols
    m, n = X.shape
    # all possible y values.have 3 valus. 
    uni_values = y.unique()

    # output, 
    # have to know the range of y if init in this way 
    p0_list=[]
    p1_list=[]
    p2_list=[]
    p_list=[p0_list, p1_list, p2_list]
    type_list=[]
    #prob of low, high, median
    p_prior=[]
    # print("m",m)
    # for y_ in uni_values:
    #     p_prior.append(  y[y==y_].value_counts()/m)
    TOTAL_LEBEL=3
    # LABEL_LIST=["low", "median","high"]
    for y_ in uni_values:        
        p_prior.append ((len(y[y == y_]) + 1) / (m + TOTAL_LEBEL) )

    # attrubute_nums = []
    k=0
    for y_ in uni_values:
        X_ = X[y == y_]
        # the total num of this class
        total_num =X_.shape[0]
        for i in range(n):
            # is continus attri? 
            X_nonclassified = X.iloc[:, i]
            Xi = X_.iloc[:, i]

   
            type_list.append(type_of_target(Xi))
            if type_of_target(Xi) == 'continuous':
                # get the min and max of Xi, then divide to bins, then calc the num of every bins 

                NUM_BINS=10
                cast_xi = pd.cut(Xi,[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
                # p for (high|ABCD)
                # not m , should be the total num of this attri
                p= (cast_xi.value_counts()+1)/(total_num+NUM_BINS)
                p_list[k].append(p)
                
            else  :
                # special handle for month , because 6,7 is missing
                if i == 1:
                    TOTAL_MON=12
                    Num_attris = TOTAL_MON
                else :
                    Num_attris = X_nonclassified.nunique()
                ret_list = []

                Xi_list = Xi.tolist()
                
                xi_min = np.min(X_nonclassified)
                # correct
                # print("xi_min", "i_", i, xi_min)
                STEP=1
                # [start, end, step]
                # for j in range(Num_attris):
                for j in range(xi_min, xi_min+Num_attris, STEP):
                    num_attri = Xi_list.count(j)
                    # print("X_nonclassified",X_nonclassified)
                    # print("i_",i, "j_",j, "count_", num_attri,"Num_attris", Num_attris )
                    p = (num_attri+1)/(total_num+Num_attris)
                    ret_list.append(p)
                # how handle output， list to series
                # but index  is consitant or not?, 
                # default index is 0,1,2,...
                # assign the index start point
                p_list[k].append(pd.Series(ret_list, index =list(range(xi_min, xi_min+Num_attris, STEP))))
                                    
        
        k=k+1
        # p0 , p
    return p_prior, p_list, type_list


# assume global value: num_prior =3
# assume global the name of this prior/label {low, median , high}
def predict_eval(p_prior, p_list, X, y):
    m, n = X.shape
    # ret=[]
    acc=0
    for i in range(m):
        # 'numpy.float64' object cannot be interpreted as an integer
        Xi=X.iloc[i, :].astype(object)
      
        perdict = predict_single(p_prior, p_list, Xi)
        if(perdict==y[i]):
            acc+=1 
    acc_p = acc/len(X)       
    # print("Acc :",acc/len(X))
    return acc_p
    # output label
def predict_single(p_prior, p_list, X):
    n = X.shape[0] #len(X)
    # X.shape 1, n 
    # 10
    # print(n)
    x_p0 ,x_p1, x_p2=0,0,0

    x_list=[x_p0, x_p1,x_p2]
    NUM_P_LIST=3
    for i in range(NUM_P_LIST):
        # err
        x_list[i] =np.log(p_prior[i]) 
        for j in range(n):

            # 3*10
            x_list[i]+=np.log(p_list[i][j][X[j]])
            # print( j,"_ ", X[j])
        x_list[i]=x_list[i].tolist()
    return LABEL_LIST[x_list.index(np.max(x_list))]





# The functions for tran_ncb, predict_ncb(GAUSSIAN)

In [45]:
def calc_condi_prob_gaussian(X, y):
    # rows and cols
    m, n = X.shape
    # all possible y values.have 3 valus. 
    uni_values = y.unique()

    # output, 
    # have to know the range of y if init in this way 
    p0_list=[]
    p1_list=[]
    p2_list=[]
    p_list=[p0_list, p1_list, p2_list]
    type_list=[]
    #prob of low, high, median
    p_prior=[]
    # print("m",m)
    # for y_ in uni_values:
    #     p_prior.append(  y[y==y_].value_counts()/m)
    TOTAL_LEBEL=3
    # LABEL_LIST=["low", "median","high"]
    for y_ in uni_values:        
        p_prior.append ((len(y[y == y_]) + 1) / (m + TOTAL_LEBEL) )

    # attrubute_nums = []
    k=0
    for y_ in uni_values:
        X_ = X[y == y_]
        # the total num of this class
        total_num =X_.shape[0]
        for i in range(n):
            # is continus attri? 
            X_nonclassified = X.iloc[:, i]
            Xi = X_.iloc[:, i]

   
            type_list.append(type_of_target(Xi))
            if type_of_target(Xi) == 'continuous':
             
                xi_mean = np.mean(Xi)
                xi_var = np.var(Xi)
                p_list[k].append( [xi_mean, xi_var])
                 
            else  :
                # special handle for month , because 6,7 is missing
                if i == 1:
                    TOTAL_MON=12
                    Num_attris = TOTAL_MON
                else :
                    Num_attris = X_nonclassified.nunique()
                ret_list = []

                Xi_list = Xi.tolist()
                
                xi_min = np.min(X_nonclassified)
                # correct
                # print("xi_min", "i_", i, xi_min)
                STEP=1
                # [start, end, step]
                # for j in range(Num_attris):
                for j in range(xi_min, xi_min+Num_attris, STEP):
                    num_attri = Xi_list.count(j)
                    # print("X_nonclassified",X_nonclassified)
                    # print("i_",i, "j_",j, "count_", num_attri,"Num_attris", Num_attris )
                    p = (num_attri+1)/(total_num+Num_attris)
                    ret_list.append(p)

                p_list[k].append(pd.Series(ret_list, index =list(range(xi_min, xi_min+Num_attris, STEP))))
                                    
        
        k=k+1
        # p0 , p
    return p_prior, p_list, type_list

def predict_eval_gaussian(p_prior, p_list, type_list, X, y):
    m, n = X.shape
    predict_list=[]
    acc=0
    for i in range(m):
        # 'numpy.float64' object cannot be interpreted as an integer
        Xi=X.iloc[i, :].astype(object)

        predict, _ = predict_single_gaussian(p_prior, p_list, type_list, Xi)
        if(predict==y[i]):
            acc+=1   
        predict_list.append(predict)
    acc_p = acc/len(X)         
    # print("Acc :",acc/len(X))
    return acc_p, predict_list
    # output label
def predict_single_gaussian(p_prior, p_list, type_list,  X):
    n = X.shape[0] #len(X)
    # X.shape 1, n 
    # 10|
    # print(n)
    x_p0 ,x_p1, x_p2=0,0,0
    
    x_list=[x_p0, x_p1,x_p2]
    NUM_P_LIST=3
    for i in range(NUM_P_LIST):
        # err
        x_list[i] =np.log(p_prior[i]) 
        for j in range(n):
            if(type_list[j]=="continuous"):
                mean, var = p_list[i][j]
                # np.log(0)
                x_list[i]+=np.log(1/np.sqrt(2*np.pi)*var * np.exp(-(X[j]-mean)**2/(2*var**2)))
            # mean0, var0 = p0_xi.conditional_pro
            # x_p1 += np.log(1 / (np.sqrt(2 * np.pi) * var1) * np.exp(- (x[i] - mean1) ** 2 / (2 * var1 ** 2)))
            # x_p0 += np.log(1 / (np.sqrt(2 * np.pi) * var0) * np.exp(- (x[i] - mean0) ** 2 / (2 * var0 ** 2)))
            # 3*10
            else:
                x_list[i]+=np.log(p_list[i][j][X[j]])
            # print( j,"_ ", X[j])
        x_list[i]=x_list[i].tolist()
    return LABEL_LIST[x_list.index(np.max(x_list))], x_list
# 


# Train and test ncb

In [55]:
import pandas as pd 
import numpy as np 
from sklearn.utils.multiclass import type_of_target
train_file_name=  '/home/aistudio/work/data/training sample.csv'
test_file_name=  '/home/aistudio/work/data/testing sample.csv'
another_test_file_name=  '/home/aistudio/work/data/another_testing sample.csv'
train_data= pd.read_csv(train_file_name)
test_data= pd.read_csv(test_file_name)
another_test_data= pd.read_csv(another_test_file_name)

train_X =train_data.iloc[:, :-1]
train_y =train_data.iloc[:, -1]
test_X =test_data.iloc[:, :-1]
test_y =test_data.iloc[:, -1]
a_test_X =another_test_data.iloc[:, :-1]
a_test_y =another_test_data.iloc[:, -1]
# print("X", X,y)

######  testing decrete algo#####

# print("#####For class low: ,", P_list[0])
# print("#####For class median: ,", P_list[1])
# print("#####For class high: ,", P_list[2])
# ret = predict_eval(P_prior, P_list,  test_X, test_y)
# print(ret)

######  testing gaussian#####

# ret = predict_eval_gaussian(P_prior, P_list, type_list, test_X, test_y)

# Part 1 
ACC_FORMAT="The accuracy on training data is __%r__. The accuracy on testing data is __%r___. "
P_prior, P_list, type_list= calc_condi_prob(train_X, train_y)
acc_train = predict_eval(P_prior, P_list, train_X, train_y)
acc_test = predict_eval(P_prior, P_list, test_X, test_y)
print("####Using discretization for continuous value: ")
print(ACC_FORMAT%(acc_train,acc_test))

P_prior_g, P_list_g, type_list_g= calc_condi_prob_gaussian(train_X, train_y)
acc_train_g,_ = predict_eval_gaussian(P_prior_g, P_list_g, type_list_g, train_X, train_y)
acc_test_g,_ = predict_eval_gaussian(P_prior_g, P_list_g, type_list_g, test_X, test_y)
print("####Using gaussian for continuous value: ")
print(ACC_FORMAT%(acc_train_g,acc_test_g))
print("####testing another file")
print("another_file data: ", another_test_data)
acc_test_g, predict_list = predict_eval_gaussian(P_prior_g, P_list_g, type_list_g, a_test_X, a_test_y)
print("#####predict result is :",predict_list)




####Using discretization for continuous value: 
The accuracy on training data is __0.6020408163265306__. The accuracy on testing data is __0.5874125874125874___. 
####Using gaussian for continuous value: 
The accuracy on training data is __0.5085034013605442__. The accuracy on testing data is __0.5314685314685315___. 
####testing another file
another_file data:     season  mnth  holiday  weekday  workingday  weathersit      temp     atemp  \
0       1     1        0        6           0           2  0.344167  0.363625   
1       1     1        0        0           0           2  0.363478  0.353739   
2       1     1        0        1           1           1  0.196364  0.189405   
3       1     1        0        2           1           1  0.200000  0.212122   
4       1     1        0        4           1           1  0.204348  0.233209   

        hum  windspeed label  
0  0.805833   0.160446   low  
1  0.696087   0.248539   low  
2  0.437273   0.248309   low  
3  0.590435   0.160296  

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in log


In [81]:
#####Part2
CONDI_FORMAT=" \
P(low| sample %r)=__%r___. P(median |sample %r)=__%r__ ,P(high |sample %r)=__%r__" 
for i in range(1,6,1):
    test=test_X.iloc[i,:].astype(object)
    # predict_single_gaussian(p_prior, p_list, type_list,  X):
    _, ret = predict_single_gaussian(P_prior_g, P_list_g, type_list_g, test)
    # print(ret)
    print(CONDI_FORMAT%(i,np.exp( ret[0]),i,np.exp( ret[1]),i,np.exp( ret[2])))

 P(low| sample 1)=__6.681864833254789e-82___. P(median |sample 1)=__3.8668741831191435e-136__ ,P(high |sample 1)=__0.0__
 P(low| sample 2)=__3.2182208691747515e-94___. P(median |sample 2)=__1.3859406750641623e-201__ ,P(high |sample 2)=__0.0__
 P(low| sample 3)=__1.7990671384371054e-41___. P(median |sample 3)=__1.439285336469206e-89__ ,P(high |sample 3)=__6.303359688755155e-210__
 P(low| sample 4)=__3.0190551570940836e-22___. P(median |sample 4)=__1.8047204938654632e-69__ ,P(high |sample 4)=__5.839772852929974e-186__
 P(low| sample 5)=__5.4244797509973215e-142___. P(median |sample 5)=__1.2319975907471703e-195__ ,P(high |sample 5)=__0.0__


In [53]:
#Part 3
# output one label
# print("train_X.shape[0]",train_X.shape[0])
# LABEL_LIST
for i in range(3):
    for j in range(train_X.shape[1]):
        print("#####For  %rth attribute| Label %r "%(j,LABEL_LIST[i]))
        if type_list[j] =="continuous":
            print("###Using Gaussian: ",P_list_g[i][j])
            print("###Using discretization: ",P_list[i][j])
        else:
            print( P_list[i][j])

#####For  0th attribute| Label 'low' 
1    0.676471
2    0.139706
3    0.029412
4    0.154412
dtype: float64
#####For  1th attribute| Label 'low' 
1     0.256944
2     0.194444
3     0.138889
4     0.069444
5     0.020833
6     0.006944
7     0.006944
8     0.013889
9     0.020833
10    0.055556
11    0.062500
12    0.152778
dtype: float64
#####For  2th attribute| Label 'low' 
0    0.947761
1    0.052239
dtype: float64
#####For  3th attribute| Label 'low' 
0    0.172662
1    0.129496
2    0.165468
3    0.143885
4    0.086331
5    0.107914
6    0.194245
dtype: float64
#####For  4th attribute| Label 'low' 
0    0.41791
1    0.58209
dtype: float64
#####For  5th attribute| Label 'low' 
1    0.459259
2    0.414815
3    0.125926
dtype: float64
#####For  6th attribute| Label 'low' 
###Using Gaussian:  [0.3011963083333333, 0.013132926177256825]
###Using discretization:  (0.2, 0.3]    0.352113
(0.3, 0.4]    0.211268
(0.1, 0.2]    0.183099
(0.4, 0.5]    0.133803
(0.5, 0.6]    0.063380
(0.0, 0.1]